In [1]:
from __future__ import division

In [2]:
from IPython.core.debugger import Tracer

In [3]:
import pyparsing as pp
import regex
import sys
from itertools import izip
import itertools as it
from copy import copy, deepcopy
from collections import Sequence
import abc
import operator as op
from collections import defaultdict, namedtuple
from overrides import overrides
import inspect
from contextlib import contextmanager
from functools import partial
from numpy import inf

In [4]:
from mymeta import lift, delift, relift, LiftableFrom
from mymixin import Listenable
from mygenerators import deleteallbutone
from myobjects import Count
from mywrappers import defaultdict
import helpers_regex as hre

In [146]:
from wrapt import ObjectProxy

In [5]:
range = xrange

# Regex Implementation of Pyparsing

## generic definitions

In [6]:
class Leaf(object):
    """ this class just denotes a Leaf of a (nested) Structure
    
    these are the base elements for the functor (map) ability
    
    further Leafs with pseudorgroup structure as element will be flatten out
    in the final Structure and thus they are not visible at all
    """
    def __init__(self, leaf):
        self.leaf = leaf
        
    def map(self, func):
        self.leaf = func(self.leaf)
    
    def __str__(self):
        return str(self.leaf)
    
    def __repr__(self):
        return repr(self.leaf)

In [150]:
class Structure(Sequence):
    """ implements generic dict-list-combining structure like it is used in pyparsing.ParseResult """
    
    # Construction
    # ------------
    
    def __init__(self, initializer=None, _list=None, _dict=None):
        """either initializer or non-empty list is needed"""
        if not _list and not initializer:
            raise RuntimeError("either _list or initializer is needed")
        self._list = _list or [Leaf(initializer)] # list of sub-Structures
        self._dict = _dict or defaultdict(list)
        self.pseudo = False
        self.liftedkeys = False

    # Logic
    # -----
    
    def set_name(self, name):
        self.group(pseudo=True, liftkeys=True)
        #self._list[0] is old self before grouping, mind this is equal to self[0], however latter access is much slower
        self._extend_dict({name: [self._list[0]]})
        return self

    def group(self, wrapper=lambda x:x, pseudo=False, liftkeys=False):
        """ CAUTION: changes self inplace
        deepcopy before if you like to have a new object """
        self.pseudo = pseudo
        self.liftedkeys = liftkeys
        
        cp = copy(self)
        Structure.__init__(self,
            _list = [wrapper(cp)],
            _dict = self._lift_keys(cp._dict) if liftkeys else defaultdict(list)
        )
        return self       
    
    @staticmethod 
    def _lift_keys(_dict):
        return defaultdict(
            list,
            { k: ['lifted'] for k, v in _dict.iteritems() }
        )
                    
    def map(self, func):
        """Structure is a functor =), everything is inplace,
        
        deepcopy before if you want to have a new object"""
        #this works as also leaves have .map function and stop recursion
        for elem in self.iter_leave_leaves():
            elem.map(func)
        return self
    
    
    # general interface methods:
    # --------------------------
    
    def __iter__(self):
        for s in self._list:
            if isinstance(s, Structure):
                if s.pseudo:
                    for s2 in iter(s): # yield from in python 3.x
                        yield s2
                else:
                    yield s
            # flatten out Leaves:
            elif isinstance(s, Leaf):
                if isinstance(s.leaf, Structure) and s.leaf.pseudo:
                    for s2 in iter(s.leaf): # yield from in python 3.x
                        yield s2
                else:
                    yield s.leaf

    def iter_leave_leaves(self):
        for s in self._list:
            if isinstance(s, Structure) and s.pseudo:
                for s2 in iter(s): # yield from in python 3.x
                    yield s2
            else:
                yield s
    
    def __getitem__(self, index):
        if isinstance(index, int):
            return list(iter(self))[index]  #TODO extremely inefficient! (but won't be really needed neither)
        else:
            return list(self._dictitem_gen(index))
    
    def _dictitem_gen(self, index):
        """ checks for 'lifted' keys and in case flattens out all results """
        if index in self.keys():
            ret = self._dict[index]
            for elem in ret:
                if elem == 'lifted':
                    # now we use true underlying structure (ignoreing pseudostructures), is this 
                    # TODO: or use sefl._list directly?
                    for s in self:
                        if s.liftedkeys:
                            # yield from in python 3.x:
                            for i in s._dictitem_gen(index):
                                yield i
                else:
                    yield elem        
                    
    def __len__(self):
        return len(self._list)
                    
    def keys(self):
        return self._dict.keys()
        
    def __add__(self, other):
        base = deepcopy(self)
        base += other # (+=) == __iadd__
        return base
    
    def __call__(self, name):
        return copy(self).set_name(name)
        
    def __iadd__(self, other):
        if not isinstance(other, Structure):
            raise NotImplemented
        self._list += other._list
        self._extend_dict(other._dict)
        return self
    
    def _extend_dict(self, other_dict):
        """ extends self._dict by other_dict however keeps only one 'lifted' entry"""
        for key in other_dict:
            self._dict[key] = list(deleteallbutone('lifted',
                self._dict[key] + other_dict[key]
            ))
            
    def __str__(self):
        return "[" + ",".join(str(i) for i in self)+ "]"
    
    def __repr__(self):
        public_attr ={attr:getattr(self, attr) for attr in dir(self) if not attr.startswith("_") and attr not in dir(self.__class__)}
        if public_attr:
            return "(%s, %s, %s)"%(repr(self._list), repr(self._dict), repr(public_attr))
        else:
            return "(%s, %s,)"%(repr(self._list), repr(self._dict))

In [220]:
def recursive_structure_delift(structure, base_class=Structure):
    """ more complex delift for structure types, as the structure type itself is usually nested """
    structure.__class__ = base_class
    for s in structure._list:
        if isinstance(s, Structure):
            recursive_structure_delift(s, base_class)
        elif isinstance(s, Leaf) and isinstance(s.leaf, Structure):
            recursive_structure_delift(s.leaf, base_class)
    return structure

## parser specific definitions

In [151]:
silently_ignore = []
_MAX_INT = sys.maxint

In [221]:
class ParserElementType(Structure):
    """ abstract class capturing the interface of an arbitrary ParserElement of pyparsing """
    __metaclass__ = abc.ABCMeta
        
    def __call__(self, name, **kwargs):
        return copy(self.copy).setResultsName(name, **kwargs)
    
    def setResultsName(self, name, **kwargs):
        self.set_name(name)
        self.update_public_attributes_dict(kwargs)
        return self # TODO is this the standard behaviour?
        
    @abc.abstractmethod
    def suppress(self):
        """Suppresses the output of this C{ParserElement}; useful to keep punctuation from
           cluttering up returned output.
           
           change outer brackets, e.g. (...) or (?P<>...) or (?<>...) to non-capturing (?:...) version
        """
        raise NotImplemented
        
    @abc.abstractmethod
    def repeat(self, min=0, max=None):
        """ repititions is the main structural addition on top of the Structure-type """
        raise NotImplemented()
        
    def parseString(self, instring, parseAll=False):
        """Execute the parse expression with the given string.
        This is the main interface to the client code, once the complete
        expression has been built.

        If you want the grammar to require that the entire input string be
        successfully parsed, then set C{parseAll} to True (equivalent to ending
        the grammar with C{L{StringEnd()}}).

        Note: C{parseString} implicitly calls C{expandtabs()} on the input string,
        in order to report proper column numbers in parse actions.
        If the input string contains tabs and
        the grammar uses parse actions that use the C{loc} argument to index into the
        string being parsed, you can ensure you have a consistent view of the input
        string by:
        - calling C{parseWithTabs} on your grammar before calling C{parseString}
          (see L{I{parseWithTabs}<parseWithTabs>})
        - define your parse action using the full C{(s,loc,toks)} signature, and
          reference the input string using the parse action's C{s} argument
        - explictly expand the tabs in your input string before calling
          C{parseString}

        Return ParseResult!
        """
        return (self+StringEnd() if parseAll else self)._parseString(instring)
        
    @abc.abstractmethod
    def _parseString(self, instring):
        raise NotImplemented()

    
    def scanString(self, instring, maxMatches=_MAX_INT, overlap=False):
        """not supported: maxMatches
        
        Scan the input string for expression matches.  Each match will return the
        matching tokens, start location, and end location.  May be called with optional
        C{maxMatches} argument, to clip scanning after 'n' matches are found.  If
        C{overlap} is specified, then overlapping matches will be reported.

        Note that the start and end locations are reported relative to the string
        being parsed.  See L{I{parseString}<parseString>} for more information on parsing
        strings with embedded tabs.
        """
        i = 0
        matches = 0
        while i < len(instring) and matches < maxMatches:
            m = self.parseString(instring[i:])
            if m is not None:
                yield m
                matches += 1
                if overlap:
                    i += 1
                else:
                    i += m.end()
            else:
                i += 1


    def transformString(self, instring):
        raise NotImplementedError("with regular expressions setParseAction is not supported and thus also not transformString")
        #maybe later by using regex substitution

    def searchString(self, instring, maxMatches=_MAX_INT):
        """Another extension to C{L{scanString}}, simplifying the access to the tokens found
           to match the given parse expression.  May be called with optional
           C{maxMatches} argument, to clip searching after 'n' matches are found.
        """
        return list(self.scanString(instring))
    
    # add, iadd already defined in Structure
            
    def __or__(self, other):
        base = deepcopy(self)
        base |= other # (|=) == __iadd__
        return base
    
    @abc.abstractmethod
    def __ior__(self, other):
        raise NotImplemented()
        

class ParseResult(ObjectProxy):
    """ class capturing interface of return type of pyparsing (and more needed methods) """
    
    # class-level default value for flattening
    FLATTEN_SINGLETONS = False
    EMPTY_DEFAULT = "EMPTYLIST" #this stands for [], however as [] is mutable, its no good idea to make it the default
    
    def __init__(self, structure, span, flatten_singletons=None, empty_default="EMPTYLIST"):
        # proxy the complete structure element:
        super(ParseResult, self).__init__(structure)
        
        # transform it slightly given the additional parameters:
        # (map is a function Structure already, as the Proxy was initialized)
        if (flatten_singletons is None and ParseResult.FLATTEN_SINGLETONS) or flatten_singletons:
            self.map(ParseResult.flatten_singletons)
        
        if empty_default != "EMPTYLIST":
            self.map(self.func_change_default(empty_default))
        elif ParseResult.EMPTY_DEFAULT != "EMPTYLIST":
            self.map(self.func_change_default(ParseResult.EMPTY_DEFAULT))
            
        # general information over ParseResult:
        self.span = span
        
    def end(self):
        "returns match end for given string"
        return self.span[1]
    
    def span(self):
        "returns (match begin, match end) for given string"
        return self.span
    
    @staticmethod
    def func_change_default(empty_default):
        def change_default(leaf):
            if leaf == []: # this is the standard empty value
                return empty_default
            
            # recursive call for recursive leaves:
            elif isinstance(leaf, Structure):
                leaf.map(change_default)
            
            # in every case default to return leaf
            return leaf
        return change_default
        
    @staticmethod
    def flatten_singletons(leaf):
        """ for this to work, the leafs must already be mapped to lists
        e.g. by using ParserElement._func_parse_leaf """
        # recursive call for recursive leaves:
        if isinstance(leaf, Structure):
            leaf.map(ParseResult.flatten_singletons)
            
        if len(leaf) == 1:
            return leaf[0]
        return leaf

In [246]:
class ParserElement(ParserElementType):
    """   
    we can immitate arbitrarily complex formula directly by a single regex-string
    the output gets restructured (in linear time) to fulfil ParserElement/Structure interface
    
    not implemented: whitespaces support
    """
    
    #: repeated substructure (just a tuple)
    Repeated = namedtuple("Repeated", ["count", "struct"])
    MatchedGroup = namedtuple("Group", ["ends", "captures"])
    
    # CONSTRUCTION
    # ============

    def __init__(self, pattern=""):
        super(ParserElement, self).__init__(initializer=Count())
        self.pattern = pattern
        self._compiled = None
    
    # LOGIC
    # =====
    
    @overrides # TODO does this work - new parameter silent
    def group(self, wrapper=lambda x:x, pseudo=False, liftkeys=False, silent=None):
        # this is inplace:
        super(ParserElement, self).group(wrapper, pseudo=pseudo, liftkeys=liftkeys)
        # normal grouping is done by Structure type,
        # but silent groups are nevertheless needed for correct regex semantics:
        if silent is None:
            pass # keep old self.pattern
        elif silent:
            self.pattern = hre.silent_group(self.pattern)
        else:
            self.pattern = hre.group(self.pattern)
        return self
    
    def suppress(self):
        """Suppresses the output of this C{ParserElement}; useful to keep punctuation from
           cluttering up returned output.
           
           change all inner brackets, e.g. (...) or (?P<>...) or (?<>...) to non-capturing (?:...) version
           
           CAUTION: NOT REVERSIBLE!
        """
        self.pattern = hre.begins_not_silently_grouped.sub("(?:", self.pattern)
    
    def repeat(self, min=0, max=inf):
        """ repeat on arbitrary ParserElement """
        if min > max:
            raise RuntimeError("min <= max needed")
        # if not singleton, then add extra layer for repition:
        if not hre.singleton_group.match(self.pattern):
            # the grouping is done by wrapping into a Leaf,
            # so that we can construct a map function which does all restructuring of the regex output
            self.group(
                wrapper = lambda struct: Leaf(self.Repeated(Count(), struct)),
                pseudo = True,
                liftkeys = True,
                silent = False,
            )
        if max is inf:
            self.pattern = r"%s{%s,}+"   % (self.pattern, min)
        elif min == max:
            self.pattern = r"%s{%s}+"    % (self.pattern, min)
        else:
            self.pattern = r"%s{%s,%s}+" % (self.pattern, min, max)
    
    @staticmethod
    def _transform_match_gen(match):
        try:
            for i in it.count(1):
                yield ParserElement.MatchedGroup(match.ends(i), match.captures(i))
        except IndexError:
            pass
    
    def _parseString(self, instring):
        """starts matchin at starts of ``instring`` - no search
        
        this is copying everything beforehand"""
        
        if self._compiled is None:
            self._compiled = regex.compile(self.pattern)
        
        m = self._compiled.match(instring)
        if m is None:
            return None
        
        mymatch = list(self._transform_match_gen(m))
        
        Count.reset()
        cp = deepcopy(self)
        cp.map(self._recursive_evalcount)
        cp.map(self._func_parse_leaf(mymatch))
        #getting default structure str, repr interface (also for nested elements):
        recursive_structure_delift(cp)
        return ParseResult(cp, span=m.span())
    
    @staticmethod
    def _recursive_evalcount(leaf):
        """ evaluates all Count instances so that they refer to fixed group"""
        if isinstance(leaf, ParserElement.Repeated):
            leaf.count.eval()
            leaf.struct.map(ParserElement._recursive_evalcount)
        elif isinstance(leaf, Count):
            leaf.eval()
        return leaf
    
    @staticmethod
    def _func_parse_leaf(match):
        """
        CAUTION: for this map to work correctly,
        every Count instance must be evaluated already recursively!
        
        i.e. first map ParserElement._recursive_evalcount
        """
        
        def recursive_parse(leaf, maxend=inf):
            # recursive case:
            if isinstance(leaf, ParserElement.Repeated):
                def gen():
                    for i, end in enumerate(match[leaf.count.value].ends):
                        if end > maxend:
                            del match[leaf.count.value].ends[:i] #delete everything parsed so far
                            # captures are of no interest at all of these Repeated elements
                            # so no need to delete them
                            break
                        yield deepcopy(leaf.struct).map(partial(recursive_parse, maxend=end))
            
                # returns structure which is labeld pseudo by initial repeat method
                return reduce(op.add, gen())
            
            # base case:
            elif isinstance(leaf, Count):
                i = -1
                for i, end in enumerate(match[leaf.value].ends):
                    if end > maxend:
                        # i is now the final valid index+1
                        break
                else:
                    # no break, i.e. we currently miss the last one, OR empty loop
                    i += 1 # if nothing was done, i=0 now, giving empty list and no deletes
                
                ret = match[leaf.value].captures[:i]
                # delete everything parsed so far (both captures end ends!):
                del match[leaf.value].captures[:i]
                del match[leaf.value].ends[:i]
                return ret
            
        return recursive_parse
    
    def __iadd__(self, other):
        if isinstance(other, basestring):
            other = Token(other)
        
        Structure.__iadd__(self, other)
        self.pattern += other.pattern
        self._compiled = None
        return self
    
    def __radd__(self, other):
        if isinstance(other, basestring):
            other = Token(other)
        other += self
        return other
        
    def __ior__(self, other):
        if isinstance(other, basestring):
            other = Token(other)
        
        Structure.__iadd__(self, other)
        self.pattern += "|" + other.pattern
        self._compiled = None
        return self
    
    def __ror__(self, other):
        if isinstance(other, basestring):
            other = Token(other)
        other |= self
        return other
        
    def __str__(self):
        return "(r'%s', %s)" % (self.pattern, Structure.__str__(self))
    
    def __repr__(self):
        return "(r'%s', %s)" % (self.pattern, Structure.__repr__(self))


def Token(pattern):
    return ParserElement(pattern)

# Pyparsing-like Interface

In [247]:
def Regex(pattern, flags=0):
    """Grouped by default. flags are locally scoped and will only effect the supplied pattern, nothing more"""
    if flags:
        str_flags = decodeflags(flags)
        pattern = r"(?%s:%s)"%(str_flags, pattern)
    return Token(hre.group(pattern))


def Word(initChars, bodyChars=None, min=1, max=0, exact=0, excludeChars=None):
    """Grouped by default. not implemented kwargs: asKeyword """
    
    if max != 0 and min > max:
        raise RuntimeError("min <= max needed")

    if excludeChars:
        initChars = initChars + "--" + excludeChars
        if bodyChars:
            bodyChars = bodyChars + "--" + excludeChars

    if exact == 1 or max == 1:
        pattern = r"[%s]{1}"%(initChars)    
    elif exact > 1:
        if bodyChars:
            pattern = r"[%s]{1}[%s]{%s}"%(initChars, bodyChars, exact-1)
        else:
            pattern = r"[%s]{%s}"%(initChars, exact)
    elif max > 1:
        if bodyChars:
            pattern = r"[%s]{1}[%s]{%s,%s}"%(initChars, bodyChars, __builtin__.max(min-1,0), max-1)
        else:
            pattern = r"[%s]{%s,%s}"%(initChars, min, max)
    else: # arbitrary upper bound
        if bodyChars:
            pattern = r"[%s]{1}[%s]{%s,}"%(initChars, bodyChars, __builtin__.max(min-1,0))
        else:
            pattern = r"[%s]{%s,}"%(initChars, min)

    # group by default:
    return Token(hre.group(pattern))

        
def SkipTo(self, expr, include=False):
    """Grouped by default. not supported: ignore=None, failOn=None.
    
    Token for skipping over all undefined text until the matched expression is found.
    If C{include} is set to true, the matched expression is also parsed (the skipped text
    and matched expression are returned as a 2-element list).  The C{ignore}
    argument is used to define grammars (typically quoted strings and comments) that
    might contain false matches.
    """
    pattern = r"(?:.(?!%s))*."%(expr)
    if include:
        pattern += expr
    # group by default:
    return Token(hre.group(self.pattern))

In [248]:
def StringStart():
    """matches beginning of the text"""
    return Token(r"^")

def StringEnd():
    """matches the end of the text"""
    return Token(r"$")

def LineStart():
    """matches beginning of a line (lines delimited by \n characters)"""
    return Regex(r"^", regex.MULTILINE)

def LineEnd():
    """matches the end of a line"""
    return Regex(r"$", regex.MULTILINE)

In [249]:
def Suppress(expr):
    expr = deepcopy(expr)
    expr.suppress()
    return expr


def And(iterable):
    """__dict__ of first element will be passed through And result """
    try:
        gen = iter(iterable)
        base = next(gen) + next(gen) # once (+) to have a new element
        for expr in gen:
            base += expr
        return base
    except StopIteration: # only one element
        return next(iter(iterable))


def MatchFirst(iterable):
    """__dict__ of first element will be passed through MatchFirst result """
    try:
        gen = iter(iterable)
        base = next(gen) | next(gen) # once (|) to have a new element
        for expr in gen:
            base |= expr
        return base
    except StopIteration: # only one element
        return next(iter(iterable))
    
#Or __xor__ and Each __and__ are missing - takes more time to implement

def Optional(expr):
    cp = copy(expr)
    cp.pattern = r"%s?" % hre.ensure_grouping(cp.pattern)
    return cp

In [250]:
def Group(expr):
    g = deepcopy(expr)
    g.group()
    return g

def GroupLiftKeys(expr):
    g = deepcopy(expr)
    g.group(liftkeys=True)
    return g

In [251]:
def OneOrMore(expr):
    return Repeat(expr, min=1)
        
def ZeroOrMore(expr):
    return Repeat(expr)

def Repeat(expr, min=0, max=inf):
    expr = deepcopy(expr)
    expr.repeat(min=min, max=max)
    return expr

# Test

In [252]:
Count.reset()
w = Word("abc", exact=3)
print w
w

(r'([abc]{3})', [?])


(r'([abc]{3})', ([?], {}, {'pattern': '([abc]{3})', 'liftedkeys': False, 'pseudo': False}))

In [253]:
ww = w + Optional(w)
print ww
ww

(r'([abc]{3})([abc]{3})?', [?,?])


(r'([abc]{3})([abc]{3})?', ([?, ?], {}, {'pattern': '([abc]{3})([abc]{3})?', 'liftedkeys': False, 'pseudo': False}))

In [254]:
gww = Group(ww)
gww

(r'([abc]{3})([abc]{3})?', ([(r'([abc]{3})([abc]{3})?', ([?, ?], {}, {'pattern': '([abc]{3})([abc]{3})?', 'liftedkeys': False, 'pseudo': False}))], {}, {'pattern': '([abc]{3})([abc]{3})?', 'liftedkeys': False, 'pseudo': False}))

In [255]:
oomww = OneOrMore(gww)
print oomww
oomww

(r'(([abc]{3})([abc]{3})?){1,}+', [Repeated(count=?, struct=(r'([abc]{3})([abc]{3})?', ([(r'([abc]{3})([abc]{3})?', ([?, ?], {}, {'pattern': '([abc]{3})([abc]{3})?', 'liftedkeys': False, 'pseudo': False}))], {}, {'pattern': '([abc]{3})([abc]{3})?', 'liftedkeys': True, 'pseudo': True})))])


(r'(([abc]{3})([abc]{3})?){1,}+', ([Repeated(count=?, struct=(r'([abc]{3})([abc]{3})?', ([(r'([abc]{3})([abc]{3})?', ([?, ?], {}, {'pattern': '([abc]{3})([abc]{3})?', 'liftedkeys': False, 'pseudo': False}))], {}, {'pattern': '([abc]{3})([abc]{3})?', 'liftedkeys': True, 'pseudo': True})))], {}, {'pattern': '(([abc]{3})([abc]{3})?){1,}+', 'liftedkeys': False, 'pseudo': False}))

In [260]:
ParseResult.FLATTEN_SINGLETONS = True
ParseResult.EMPTY_DEFAULT = "EMPTYLIST"

In [261]:
data = "aaabcbccccccaac"
pr = oomww.parseString(data)

print type(pr)
print pr
pr

<class '__main__.ParseResult'>
[[aaa,bcb],[ccc,ccc],[aac,[]]]


<ParseResult at 0x7f76fd54e4d0 for Structure at 0x7f76fd55a8d0>